In [2]:
import numpy as np
import matplotlib as plt
import pandas as pd
import random
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import requests
import json
import re

In [5]:
# url_path = "https://www.land.mlit.go.jp/webland/api/TradeListSearch?from=20171&to=20184&area=13"
# request_result = requests.get(url_path)
# data_json = request_result.json()["data"]
# print(len(data_json))
# print(data_json[0])

In [10]:
# data_pd = pd.io.json.json_normalize(data_json)
# print(data_pd.shape)
# print(data_pd.head(10))
# https://www.land.mlit.go.jp/webland/api.html
# https://www.land.mlit.go.jp/webland/download.html